# AMS 528 Homework 2
**submitted by Jiaxi Zhao on 15th March, 2021**

In [35]:
import numpy as np
import numpy.linalg as alg
import sympy as sp
from scipy.linalg import solve as ss
from sympy import *
from matplotlib import pyplot as plt 
%matplotlib inline

## Diffusion equation with Dirichlet BC
Consider the following IBVP for the diffusion equation
\begin{equation}
    \begin{aligned}
        v_t & = v_{xx},    \\
        v(0; t) & = v(1; t) = 0     \\ 
        v(x; 0) & = \sin(3\pi x).
    \end{aligned}
\end{equation}
The exact solution is
\begin{equation}
    v(x; t) = \exp(-9\pi^2 t) \sin(3\pi x).
\end{equation}

### BTCS scheme
We use the BTCS scheme and obtain numerical solutions at time $t = 0.02$ using the number of numerical points N and time steps as in the tables below. The numerical scheme is given by 
\begin{equation}
    \frac{v_k^{n + 1} - v_k^n}{\Delta t} = \frac{v_{k + 1}^{n + 1} + v_{k - 1}^{n + 1} - 2v_k^{n + 1}}{( \Delta x )^2},
\end{equation}
which can be written in the following matrix form to solve a linear system
\begin{equation}
    \mathbf{Q}\mathbf{v}^{n + 1} = \mathbf{v}^n,
\end{equation}
with
\begin{equation}
    \mathbf{Q} = \begin{pmatrix}
        1 + 2R & -R & 0 & \cdots & 0 & 0       \\
            -R & 1 + 2R & -R & \cdots & 0 & 0  \\
                0 & -R & 1 + 2R & \cdots & 0 & 0 \\
                \vdots & \vdots & \vdots & \ddots & \vdots \\
                0 & 0 & 0 & \cdots & 1 + 2R & -R  \\
                0 & 0 & 0 & \cdots & -R & 1 + 2R
        \end{pmatrix}, \quad R = \frac{\Delta t}{\Delta x^2}.
\end{equation}

In [42]:
def BTCS(N = 64, Plot = False):
    
    x = np.linspace(0, 1, N + 1)
    v = np.sin(3 * np.pi * x)
    dx = 1 / N
    dt = (dx ** 2) / 4
    T  = 1 / 8
    R = dt / (dx ** 2)
    Q1 = np.zeros([N - 1, N - 1])
    for i in range(N - 1):
        Q1[i, i] = 1 + 2 * R
        if i != 0:
            Q1[i, i - 1] = -R
        if i != N - 2:
            Q1[i, i + 1] = -R

    for t in range(int(T / dt)):
        tmp = ss(Q1, v[1:N])
        v[1:N] = tmp

    v_truth = np.exp(-9 * np.pi ** 2 * T) * np.sin(3 * np.pi * x)
    error_inf = np.max(v_truth - v)
    error_l2 = alg.norm(v_truth - v) * np.sqrt(dx)

    if Plot:
        fig = plt.figure()
        ax  = fig.add_subplot(111)
        ax.plot(x, v, label = 'Numerical solution')
        ax.plot(x, v_truth, label = 'Exact solution')
        ax.set_xlabel('x')
        ax.set_ylabel('v')
        ax.set_title('Numerical solution v.s. Exact solution',fontsize=12)
        ax.legend()
        plt.show()
        
    return [error_inf, error_l2]

In [23]:
prev_dx = 1
prev_err_l2 = 1
prev_err_inf = 1

for i in range(5, 9):
    N = np.power(2, i)
    [error_l2, error_inf] = BTCS(N)
    dx = 1 / N
    R_l2 = np.log(error_l2/prev_err_l2) / np.log(dx/prev_dx)
    R_inf = np.log(error_inf/prev_err_inf) / np.log(dx/prev_dx)
    
    
    #print('<td>',format(error_l2,'.2e'),'</td>')
    #print('<td>',format(R_l2,'.2e'),'</td>')
    #print('<td>',format(error_inf,'.2e'),'</td>')
    #print('<td>',format(R_inf,'.2e'),'</td>')
    print([error_l2, error_inf, R_l2, R_inf])
    prev_dx = dx
    prev_err_l2 = error_l2
    prev_err_inf = error_inf

[0.0004017848078126029, 0.00028410476218179443, 2.256257873054726, 2.3562578730549606]
[9.811618539392469e-05, 6.937862003596124e-05, 2.033859963937118, 2.033859963940924]
[2.4384291485679468e-05, 1.7242297863912418e-05, 2.0085390914475942, 2.008539091446041]
[6.087039547368958e-06, 4.3041869410820455e-06, 2.0021394096600704, 2.0021394097281195]


<table border="1" align="center"><tbody>
    <tr>
        <th>N</th>
        <th>$l_{2, \Delta x}$ error</th>
        <th>Convergence order</th>
        <th>$l_{\infty}$ error</th>
        <th>Convergence order</th>
    </tr>
    <tr>
        <td>$32$</td>
        <td> 4.02e-04 </td>
        <td> NA </td>
        <td> 2.84e-04 </td>
        <td> NA </td>
    </tr>
    <tr>
        <td>$64$</td>
        <td> 9.81e-05 </td>
        <td> 2.03e+00 </td>
        <td> 6.94e-05 </td>
        <td> 2.03e+00 </td>
    </tr>
    <tr>
        <td>$128$</td>
        <td> 2.44e-05 </td>
        <td> 2.01e+00 </td>
        <td> 1.72e-05 </td>
        <td> 2.01e+00 </td>
    </tr>
    <tr>
        <td>$256$</td>
        <td> 6.09e-06 </td>
        <td> 2.00e+00 </td>
        <td> 4.30e-06 </td>
        <td> 2.00e+00 </td>
    </tr>
</table> 

### Crank-Nicolson scheme

In [25]:
def CN(N = 64, Plot = False):
    
    x = np.linspace(0, 1, N + 1)
    v = np.sin(3 * np.pi * x)
    dx = 1 / N
    dt = (dx ** 2) / 4
    T  = 1 / 16
    R = dt / (dx ** 2) / 2
    Q1 = np.zeros([N - 1, N - 1])
    Q2 = np.zeros([N - 1, N - 1])
    for i in range(N - 1):
        Q1[i, i] = 1 + 2 * R
        Q2[i, i] = 1 - 2 * R
        if i != 0:
            Q1[i, i - 1] = -R
            Q2[i, i - 1] = R
        if i != N - 2:
            Q1[i, i + 1] = -R
            Q2[i, i + 1] = R
    for t in range(int(T / dt)):
        tmp = ss(Q1, Q2 @ v[1:N])
        v[1:N] = tmp
    v_truth = np.exp(-9 * np.pi ** 2 * T) * np.sin(3 * np.pi * x)
    error_inf = np.max(v_truth - v)
    error_l2 = alg.norm(v_truth - v) * np.sqrt(dx)

    if Plot:
        fig = plt.figure()
        ax  = fig.add_subplot(111)
        ax.plot(x, v, label = 'Numerical solution')
        ax.plot(x, v_truth, label = 'Exact solution')
        ax.set_xlabel('x')
        ax.set_ylabel('v')
        ax.set_title('Numerical solution v.s. Exact solution',fontsize=12)
        ax.legend()
        plt.show()
    
    return [error_inf, error_l2]

In [27]:
prev_dx = 1
prev_err_l2 = 1
prev_err_inf = 1

for i in range(5, 9):
    N = np.power(2, i)
    [error_l2, error_inf] = CN(N)
    dx = 1 / N
    R_l2 = np.log(error_l2/prev_err_l2) / np.log(dx/prev_dx)
    R_inf = np.log(error_inf/prev_err_inf) / np.log(dx/prev_dx)
    
    
    #print('<td>',format(error_l2,'.2e'),'</td>')
    #print('<td>',format(R_l2,'.2e'),'</td>')
    #print('<td>',format(error_inf,'.2e'),'</td>')
    #print('<td>',format(R_inf,'.2e'),'</td>')
    print([error_l2, error_inf, R_l2, R_inf])
    prev_dx = dx
    prev_err_l2 = error_l2
    prev_err_inf = error_inf

[0.0001575906123139634, 0.00011143339061823702, 2.5263061567320957, 2.62630615673289]
[3.9052334089665144e-05, 2.7614170255776346e-05, 2.0127009124743362, 2.0127009124802107]
[9.741599975609302e-06, 6.888351402195895e-06, 2.003178130932132, 2.003178130956665]
[2.4340588019484966e-06, 1.7211394844665083e-06, 2.000794721183901, 2.0007947213155544]


<table border="1" align="center"><tbody>
    <tr>
        <th>N</th>
        <th>$l_{2, \Delta x}$ error</th>
        <th>Convergence order</th>
        <th>$l_{\infty}$ error</th>
        <th>Convergence order</th>
    </tr>
    <tr>
        <td>$32$</td>
        <td> 1.58e-04 </td>
        <td> NA </td>
        <td> 1.11e-04 </td>
        <td> NA </td>
    </tr>
    <tr>
        <td>$64$</td>
        <td> 3.91e-05 </td>
        <td> 2.01e+00 </td>
        <td> 2.76e-05 </td>
        <td> 2.01e+00 </td>
    </tr>
    <tr>
        <td>$128$</td>
        <td> 9.74e-06 </td>
        <td> 2.00e+00 </td>
        <td> 6.89e-06 </td>
        <td> 2.00e+00 </td>
    </tr>
    <tr>
        <td>$256$</td>
        <td> 2.43e-06 </td>
        <td> 2.00e+00 </td>
        <td> 1.72e-06 </td>
        <td> 2.00e+00 </td>
    </tr>
</table> 